In [1]:
using Revise
!(pwd() in LOAD_PATH) ? push!(LOAD_PATH, pwd()) : nothing
include("SISEPUEDEPJSF.jl")
import .SISEPUEDEPJSF

ENV["COLUMNS"] = 240


using Pkg
Pkg.activate(".")


using DataFrames
using NemoMod
using SQLite
using Cbc
using JuMP
using Gurobi
using GAMS

include("setup_analysis.jl")

  Activating project at `~/Documents/Projects/git_jbus/sisepuede/julia`
[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72]
  ** incremental compilation may be fatally broken for this module **

[ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
[ Info: Precompiling GAMS [1ca51c6a-1b4d-4546-9ae1-53e0a243ab12]
[ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]


"/Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database.sqlite"

In [26]:
fp_sqlite = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database.sqlite"
fp_sqlite_orig = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database_ORIG.sqlite"

# USE TO REINITIALIZE fp_sqlite
# cp(fp_sqlite_orig, fp_sqlite)
#

"/Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database_ORIG.sqlite"

In [100]:
#cp(fp_sqlite, fp_sqlite_orig, force = true)


In [27]:
db = SQLite.DB(fp_sqlite)
db_orig = SQLite.DB(fp_sqlite_orig)

SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database.sqlite")

In [28]:
dict_tables = Dict()

tabs = [x.name for x in SQLite.tables(db)]
table_name = 0
tabs_populated = Vector{String}()
for k in 1:length(tabs)
    table_name = tabs[k]
    df_tmp = DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
    dict_tables[table_name] = df_tmp
    
    (nrow(df_tmp) > 0) ? push!(tabs_populated, table_name) : nothing
end


tabs = sort([x for x in tabs_populated if (x != uppercase(x)) & !startswith(x, "v")])
println.(tabs);

CapacityFactor
CapitalCost
CapitalCostStorage
DefaultParams
EmissionActivityRatio
FixedCost
InputActivityRatio
LTsGroup
MinShareProduction
MinStorageCharge
OperationalLife
OperationalLifeStorage
OutputActivityRatio
REMinProductionTarget
RETagTechnology
ReserveMargin
ReserveMarginTagFuel
ReserveMarginTagTechnology
ResidualCapacity
ResidualStorageCapacity
SpecifiedAnnualDemand
SpecifiedDemandProfile
StorageLevelStart
TechnologyFromStorage
TechnologyToStorage
TotalAnnualMaxCapacityInvestment
TotalAnnualMaxCapacityInvestmentStorage
TotalTechnologyAnnualActivityLowerLimit
TotalTechnologyAnnualActivityUpperLimit
VariableCost
Version
YearSplit
sqlite_sequence


In [30]:
df = dict_tables["SpecifiedAnnualDemand"]
filter((x -> (x[:y] in ["1000", "1001"])), df)

Row,id,r,f,y,val
,Int64,String,String,String,Float64
1,1,colombia,fuel_coal,1000,2094.44
2,2,colombia,fuel_coal,1001,2382.55
3,37,colombia,fuel_crude,1000,1721.36
4,38,colombia,fuel_crude,1001,1369.58
5,73,colombia,fuel_diesel,1000,488.131
6,74,colombia,fuel_diesel,1001,532.713
7,109,colombia,fuel_electricity,1000,664.616
8,110,colombia,fuel_electricity,1001,667.966
9,145,colombia,fuel_gasoline,1000,101.87


In [34]:
using HiGHS

[ Info: Precompiling HiGHS [87dc4568-4c63-4d18-b0c0-bb2238e4078b]


In [110]:
SQLite.DBInterface.execute(db, "delete from TotalAnnualMaxCapacity;");
# REMOVED:
# MinShareProduction
# TotalTechnologyAnnualActivityUpperLimit
# TotalAnnualMaxCapacityInvestment
# REMinProductionTarget
# TotalAnnualMaxCapacityInvestmentStorage
# TotalTechnologyAnnualActivityLowerLimit
# TotalAnnualMaxCapacity

In [101]:
model = Model(Gurobi.Optimizer); #set_silent(model)
@time NemoMod.calculatescenario(
    fp_sqlite_orig; 
    #calcyears = collect(1000:1035),
    jumpmodel = model,
    numprocs = 1,
    varstosave = "vdemandnn, vnewcapacity, vtotalcapacityannual, vproductionbytechnologyannual, vproductionnn, vusebytechnologyannual, vusenn, vtotaldiscountedcost, vannualtechnologyemission"
)




2023-11-Jul 22:24:49.485 Started modeling scenario.
2023-11-Jul 22:24:49.490 Validated run-time arguments.
2023-11-Jul 22:24:49.493 Connected to scenario database. Path = /Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database_ORIG.sqlite.
2023-11-Jul 22:24:49.597 Dropped pre-existing result tables from database.
2023-11-Jul 22:24:49.599 Verified that transmission modeling is not enabled.
2023-11-Jul 22:24:49.876 Created parameter views and indices.
2023-11-Jul 22:24:49.892 Created temporary tables.
2023-11-Jul 22:24:53.261 Executed core database queries.
2023-11-Jul 22:24:53.266 Defined dimensions.
2023-11-Jul 22:24:53.285 Defined demand variables.
2023-11-Jul 22:24:53.300 Defined storage variables.
2023-11-Jul 22:24:53.302 Defined capacity variables.
2023-11-Jul 22:24:55.212 Defined activity variables.
2023-11-Jul 22:24:55.218 Defined costing variables.
2023-11-Jul 22:24:55.219 Defined reserve margin variables.
2023-11-Jul 22:24:55.250 Defined emissions v

OPTIMAL::TerminationStatusCode = 1

In [52]:
#println.(sort([x.name for x in SQLite.tables(db)]))
sum(filter(x -> (x[:y] in ["1000", "1001", "1002"]), df_tmp)[:, :val])

130.12591369100446

In [102]:
df_tmp2 = DBInterface.execute(db, "select * from vproductionbytechnologyannual") |> DataFrame;

In [91]:
sum(filter(x -> ((x[:y] in ["1002"]) & true), df_tmp2)[:, :val])

1.0000010712401937e10

In [104]:
filter(x -> ((x[:y] in ["1001"]) & occursin("supply_fuel", x[:t])), df_tmp2)

Row,r,t,f,y,val,solvedtm
,String,String,String,String,Float64,String
1,colombia,supply_fuel_wind,fuel_wind,1001,0.590284,2023-07-11 21:51:28.464
2,colombia,supply_fuel_diesel,fuel_diesel,1001,141.678,2023-07-11 21:51:28.464
3,colombia,supply_fuel_kerosene,fuel_kerosene,1001,25.922,2023-07-11 21:51:28.464
4,colombia,supply_fuel_solar,fuel_solar,1001,0.104328,2023-07-11 21:51:28.464
5,colombia,supply_fuel_coal_deposits,fuel_coal_deposits,1001,2540.38,2023-07-11 21:51:28.464
6,colombia,supply_fuel_electricity,fuel_electricity,1001,5.35107,2023-07-11 21:51:28.464
7,colombia,supply_fuel_oil,fuel_oil,1001,120.91,2023-07-11 21:51:28.464
8,colombia,supply_fuel_natural_gas,fuel_natural_gas,1001,0.434865,2023-07-11 21:51:28.464
9,colombia,supply_fuel_hydropower,fuel_hydropower,1001,565.065,2023-07-11 21:51:28.464


In [37]:
dict_tables = Dict()
db = SQLite.DB(fp_sqlite)

tabs = [x.name for x in SQLite.tables(db)]
table_name = 0
tabs_populated = Vector{String}()
for k in 1:length(tabs)
    table_name = tabs[k]
    df_tmp = DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
    dict_tables[table_name] = df_tmp
    
    (nrow(df_tmp) > 0) ? push!(tabs_populated, table_name) : nothing
end


tabs = sort([x for x in tabs_populated if (x != uppercase(x)) & !startswith(x, "v")])
println.(tabs);

CapacityFactor
CapitalCost
CapitalCostStorage
DefaultParams
EmissionActivityRatio
FixedCost
InputActivityRatio
LTsGroup
MinShareProduction
MinStorageCharge
OperationalLife
OperationalLifeStorage
OutputActivityRatio
REMinProductionTarget
RETagTechnology
ReserveMargin
ReserveMarginTagFuel
ReserveMarginTagTechnology
ResidualCapacity
ResidualStorageCapacity
SpecifiedAnnualDemand
SpecifiedDemandProfile
StorageLevelStart
TechnologyFromStorage
TechnologyToStorage
TotalAnnualMaxCapacityInvestment
TotalAnnualMaxCapacityInvestmentStorage
TotalTechnologyAnnualActivityLowerLimit
TotalTechnologyAnnualActivityUpperLimit
VariableCost
Version
YearSplit
sqlite_sequence


In [100]:
#dict_tables["SpecifiedAnnualDemand"][:, :val]
dict_tables["SpecifiedAnnualDemand"]

360-element Vector{Float64}:
 343.40338999388996
 166.260743240093
 317.57131828315437
 287.0634718536966
 348.50913428029935
 141.7188064967662
 143.84129468389045
 145.9714552970422
 148.11439856594149
 150.26794575662922
 152.4300156623278
 157.38204483949866
 162.3931849352637
   ⋮
 681.0166325448566
 694.930493915321
 709.9013470984116
 725.2084700921108
 740.8552346253726
 756.8455968452607
 773.1840637666146
 790.8844535054608
 808.9589037005766
 827.4138441791765
 846.2561859329517
 865.4933320302162

In [85]:
df_tmp = DBInterface.execute(db, "select * from MinShareProduction") |> DataFrame;
sum(filter(x -> ((x[:y] == "1006") & (x[:f] == "fuel_electricity")), df_tmp)[:, :val])

0.9933727066246939

In [86]:
tab_name = "MinShareProduction"
#df_tmp = DBInterface.execute(db, "select * from MinShareProduction") |> DataFrame;
# & (x[:f] in ["fuel_electricity"])
df = filter(x -> ((x[:y] == "1006")), get(dict_tables, tab_name, nothing))
#combine(groupby(df[:, [:t, :f, :val]], [:f]), :val => sum)

,id,r,t,f,y,val
,Int64,String,String,String,String,Float64
1,7,mexico,pp_coal,fuel_electricity,1006,0.0401529
2,43,mexico,pp_gas,fuel_electricity,1006,0.588903
3,79,mexico,pp_geothermal,fuel_electricity,1006,0.0156325
4,115,mexico,pp_hydropower,fuel_electricity,1006,0.112589
5,151,mexico,pp_nuclear,fuel_electricity,1006,0.0367622
6,187,mexico,pp_oil,fuel_electricity,1006,0.150896
7,223,mexico,pp_solar,fuel_electricity,1006,0.0403472
8,259,mexico,supply_fuel_coal,fuel_coal,1006,0.389911
9,295,mexico,supply_fuel_crude,fuel_crude,1006,0.0


In [105]:
dict_tables


CapacityFactor
CapitalCost
CapitalCostStorage
DefaultParams
EmissionActivityRatio
FixedCost
InputActivityRatio
LTsGroup
MinStorageCharge
OperationalLife
OperationalLifeStorage
OutputActivityRatio
RETagTechnology
ReserveMargin
ReserveMarginTagFuel
ReserveMarginTagTechnology
ResidualCapacity
ResidualStorageCapacity
SpecifiedAnnualDemand
SpecifiedDemandProfile
StorageLevelStart
TechnologyFromStorage
TechnologyToStorage
TotalAnnualMaxCapacity
TotalAnnualMaxCapacityInvestment
VariableCost
Version
YearSplit
sqlite_sequence


In [24]:
df_tmp = DBInterface.execute(db, "select * from TotalTechnologyAnnualActivityLowerLimit") |> DataFrame;
filter(x -> (x[:y] in ["1005", "1006", "1035"]), df_tmp)

,id,r,t,y,val
,Int64,String,String,String,Float64
1,6,brazil,pp_biogas,1005,0.0
2,7,brazil,pp_biogas,1006,0.0
3,36,brazil,pp_biogas,1035,0.0
4,62,brazil,pp_hydropower,1035,1857.23
5,68,brazil,pp_waste_incineration,1005,0.0
6,69,brazil,pp_waste_incineration,1006,0.0
7,98,brazil,pp_waste_incineration,1035,0.0


In [146]:
tables_check = []
for k in 1:length(tabs)
    table_name = tabs[k]
    
    (
        ("y" in names(dict_tables[table_name])) &
        (nrow(dict_tables[table_name]) > 0)
    ) ? push!(tables_check, table_name) : nothing
end

In [147]:
tables_check

19-element Vector{Any}:
 "YearSplit"
 "VariableCost"
 "TotalAnnualMaxCapacityInvestmentStorage"
 "SpecifiedDemandProfile"
 "SpecifiedAnnualDemand"
 "ResidualStorageCapacity"
 "ResidualCapacity"
 "ReserveMarginTagTechnology"
 "ReserveMarginTagFuel"
 "ReserveMargin"
 "RETagTechnology"
 "OutputActivityRatio"
 "MinStorageCharge"
 "InputActivityRatio"
 "FixedCost"
 "EmissionActivityRatio"
 "CapitalCostStorage"
 "CapitalCost"
 "CapacityFactor"

In [186]:
df = get(dict_tables, "REMinProductionTarget", nothing)
df
#filter(x -> (x[:y] == "1034"), df)




,id,r,f,y,val
,Int64,String,String,String,Float64
1,1,brazil,fuel_electricity,1000,0.0
2,2,brazil,fuel_electricity,1001,0.0
3,3,brazil,fuel_electricity,1002,0.0
4,4,brazil,fuel_electricity,1003,0.0
5,5,brazil,fuel_electricity,1004,0.0
6,6,brazil,fuel_electricity,1005,0.0
7,7,brazil,fuel_electricity,1006,0.0
8,8,brazil,fuel_electricity,1007,0.0
9,9,brazil,fuel_electricity,1008,0.0


In [95]:
df = get(dict_tables, "SpecifiedAnnualDemand", nothing)
filter(x -> (x[:y] == "1035"), df)




,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,36,brazil,fp_hydrogen_electrolysis,fuel_electricity,generate,1035,1.56127
2,72,brazil,fp_hydrogen_gasification,fuel_coal,generate,1035,1.967
3,108,brazil,fp_hydrogen_reformation,fuel_natural_gas,generate,1035,1.315
4,144,brazil,fp_natural_gas,fuel_electricity,generate,1035,0.0106328
5,180,brazil,fp_natural_gas,fuel_natural_gas_unprocessed,generate,1035,1.0
6,216,brazil,fp_petroleum_refinement,fuel_crude,generate,1035,1.08696
7,252,brazil,me_coal,fuel_coal_deposits,generate,1035,1.0023
8,288,brazil,me_coal,fuel_diesel,generate,1035,0.00782609
9,324,brazil,me_coal,fuel_electricity,generate,1035,0.00882271


In [68]:
println.(sort(collect(keys(dict_tables))));

AccumulatedAnnualDemand
AnnualEmissionLimit
AnnualExogenousEmission
AvailabilityFactor
CapacityFactor
CapacityOfOneTechnologyUnit
CapacityToActivityUnit
CapitalCost
CapitalCostStorage
DefaultParams
DepreciationMethod
DiscountRate
EMISSION
EmissionActivityRatio
EmissionsPenalty
FUEL
FixedCost
InputActivityRatio
InterestRateStorage
InterestRateTechnology
LTsGroup
MODE_OF_OPERATION
MinShareProduction
MinStorageCharge
MinimumUtilization
ModelPeriodEmissionLimit
ModelPeriodExogenousEmission
NODE
NodalDistributionDemand
NodalDistributionStorageCapacity
NodalDistributionTechnologyCapacity
OperationalLife
OperationalLifeStorage
OutputActivityRatio
REGION
REMinProductionTarget
RETagTechnology
RampRate
RampingReset
ReserveMargin
ReserveMarginTagFuel
ReserveMarginTagTechnology
ResidualCapacity
ResidualStorageCapacity
STORAGE
SpecifiedAnnualDemand
SpecifiedDemandProfile
StorageFullLoadHours
StorageLevelStart
StorageMaxChargeRate
StorageMaxDischargeRate
TECHNOLOGY
TIMESLICE
TSGROUP1
TSGROUP2
Techno

In [44]:
#df_max_investment = get(dict_tables, "TotalAnnualMaxCapacityInvestment", nothing)
yrs_keep = [string(x) for x in 1000:1034]
filter!(x -> ((x[:y] in yrs_keep) | (x[:t] == "pp_biomass")), df_max_investment)
SQLite.load!(df_max_investment, db, "TotalAnnualMaxCapacityInvestment"; replace = true)

"TotalAnnualMaxCapacityInvestment"

In [70]:
SQLite.load!(df_max_investment, db, "TotalAnnualMaxCapacityInvestment"; replace = true)



"TotalAnnualMinCapacity"

In [77]:
df_view = get(dict_tables, "OutputActivityRatio", nothing)

filter(x -> ((x[:y] == "1035") & (x[:f] == "fuel_hydrogen")), df_view)

,id,r,t,f,m,y,val
,Int64,String,String,String,String,String,Float64
1,36,brazil,fp_hydrogen_electrolysis,fuel_hydrogen,generate,1035,1.0
2,72,brazil,fp_hydrogen_gasification,fuel_hydrogen,generate,1035,1.0
3,108,brazil,fp_hydrogen_reformation,fuel_hydrogen,generate,1035,1.0
4,1656,brazil,supply_fuel_hydrogen,fuel_hydrogen,generate,1035,1.0


In [10]:
df_view = get(dict_tables, "MinShareProduction", nothing)

filter(x -> (x[:y] == "1034"), df_view)

,id,r,t,f,y,val
,Int64,String,String,String,String,Float64
1,35,brazil,fp_hydrogen_electrolysis,fuel_hydrogen,1034,0.889205
2,71,brazil,pp_coal,fuel_electricity,1034,0.0
3,107,brazil,pp_gas,fuel_electricity,1034,0.0
4,143,brazil,pp_geothermal,fuel_electricity,1034,0.0855676
5,179,brazil,pp_hydropower,fuel_electricity,1034,0.565988
6,215,brazil,pp_nuclear,fuel_electricity,1034,0.0
7,251,brazil,pp_oil,fuel_electricity,1034,0.0
8,287,brazil,pp_solar,fuel_electricity,1034,0.128803
9,323,brazil,pp_wind,fuel_electricity,1034,0.131397


In [63]:
for k in sort(collect(keys(dict_tables)))
    
    (string(k[1:1]) == "v") ? println(get(dict_tables, k, nothing)) : nothing
end

0×6 DataFrame
0×5 DataFrame
0×6 DataFrame
0×6 DataFrame
0×5 DataFrame
0×4 DataFrame
0×6 DataFrame
0×6 DataFrame


In [48]:
#df = get(dict_tables, "CapacityFactor", nothing)
#filter(x -> ((x[:y] == "1000") & (x[:t] == "pp_hydropower")), df)
#DBInterface.execute(db, "drop * from SpecifiedDemandProfile")
table_name = "SpecifiedDemandProfile"
DBInterface.execute(db, "delete from $(table_name)")

SQLite.Query(SQLite.Stmt(SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite"), 1), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol, Int64}(), Base.RefValue{Int64}(0))

In [36]:
#keys(dict_tables)
for tab in sort(collect(keys(dict_tables)))
    
    df = get(dict_tables, tab, nothing)
    #if "r" in names(df)
    #    println(tab)
    #    all_regions = Set(df[:, :r])
    #    println("regions:\t\n$(all_regions)")
    #    (nrow(df) == 0) ? println("**NO ROWS**\n\n") : println("\n\n")
    
    if "id" in names(df)
        s_id = Set(df[:, :id])
        (length(s_id) != nrow(df)) ? println("ID MISTMATCH IN $(tab)") : nothing
    end
    
end



In [18]:
get(dict_tables, "vtotalcapacityannual", nothing)

,r,t,y,val,solvedtm
,String?,String?,String?,Float64?,String?


In [126]:
filter(x -> (x[:f] != "fuel_electricity"), get(dict_tables, "SpecifiedAnnualDemand", nothing))

,id,r,f,y,val
,Int64,String,String,String,Float64
1,4,costa_rica,fuel_oil,1000,72.0
2,5,costa_rica,fuel_oil,1001,73.5
3,6,costa_rica,fuel_oil,1002,75.0


In [99]:
sort([x for x in keys(dict_tables) if String(x[1:1]) == "v"])


8-element Vector{String}:
 "vdemandnn"
 "vnewcapacity"
 "vproductionbytechnologyannual"
 "vproductionnn"
 "vtotalcapacityannual"
 "vtotaldiscountedcost"
 "vusebytechnologyannual"
 "vusenn"

In [127]:
get(dict_tables, "MinShareProduction", nothing)

,id,r,t,f,y,val
,Int64,String,String,String,String,Float64
1,1,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1000,0.0
2,2,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1001,0.15
3,3,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1002,0.3


In [130]:
t1 = get(dict_tables, "vproductionbytechnologyannual", nothing)
#filter(x -> (x[:t] == "pp_oil"), t1)
filter(x -> (x[:y] == "1002"), t1)


,r,t,f,y,val
,String,String,String,String,Float64
1,costa_rica,supply_fuel_crude,fuel_crude,1002,3.15576e11
2,costa_rica,pp_hydropower,fuel_electricity,1002,13.7627
3,costa_rica,pp_solar,fuel_electricity,1002,1.21603
4,costa_rica,st_compressed_air,fuel_electricity,1002,14.9759
5,costa_rica,st_batteries,fuel_electricity,1002,14.9787
6,costa_rica,supply_fuel_oil_from_fuel_electricity,fuel_oil,1002,22.5
7,costa_rica,st_flywheels,fuel_electricity,1002,14.9787
8,costa_rica,supply_fuel_solar,fuel_solar,1002,4.8255
9,costa_rica,supply_fuel_oil,fuel_oil,1002,52.5


In [139]:
(24*365*0.75*0.085)*1.298

724.8681

In [109]:
tab = filter(x -> ((x[:y] == "1002") & (x[:f] == "fuel_electricity")), t1)
sum(tab[:, :val]) - 21.7722

37.37555310931303

In [67]:
t2 = get(dict_tables, "InputActivityRatio", nothing)
filter(x -> ((x[:y] == "1000") & (x[:t] == "pp_oil")), t2)
166.262 - 36.947*2.5
37.2041*2.5

93.01024999999998

In [105]:
t3 = get(dict_tables, "SpecifiedAnnualDemand", nothing)
#filter(x -> ((x[:y] == "1001") & (x[:f] == "fuel_oil")), t3)
filter(x -> ((x[:y] == "1002")), t3)


,id,r,f,y,val
,Int64,String,String,String,Float64
1,3,costa_rica,fuel_electricity,1002,36.287
2,6,costa_rica,fuel_oil,1002,72.5739


In [88]:
i = 37
print("table_name: $(tabs_populated[i])")
df_cur = get(dict_tables, tabs_populated[i], nothing);
df_cur

table_name: CapacityFactor

,id,r,t,l,y,val
,Int64,String,String,String,String,Float64
1,1,costa_rica,pp_geothermal,fallwd0,1000,0.72
2,2,costa_rica,pp_geothermal,fallwd0,1001,0.72
3,3,costa_rica,pp_geothermal,fallwd0,1002,0.72
4,4,costa_rica,pp_geothermal,fallwd0,1003,0.72
5,5,costa_rica,pp_geothermal,fallwd0,1004,0.72
6,6,costa_rica,pp_geothermal,fallwd0,1005,0.72
7,7,costa_rica,pp_geothermal,fallwd0,1006,0.72
8,8,costa_rica,pp_geothermal,fallwd0,1007,0.72
9,9,costa_rica,pp_geothermal,fallwd0,1008,0.72


In [8]:
#DBInterface.execute(db, "select * from $(table_name)") |> DataFrame
#sort(collect(keys(dict_tables)))
#get(dict_tables, "CapacityFactor", nothing)

In [22]:
DBInterface.execute(db, "select * from CapitalCostStorage where (id in (1, 2, 3)) and (y in ('1005'))") |> DataFrame

,id,r,s,y,val
,Int64?,String?,String?,String?,Float64?


In [91]:
fp_test = normpath(joinpath(pathof(NemoMod), "..", "..", "test", "storage_test.sqlite"))


"/Users/jsyme/.julia/packages/NemoMod/vg6Tx/test/storage_test.sqlite"

In [115]:

model = Model(Gurobi.Optimizer)
#set_optimizer_attribute(model, "Solver", "cplex")
@time NemoMod.calculatescenario(
    fp_sqlite_nemomod_db_tmp; 
    jumpmodel = model,
    numprocs = 1
)

2022-31-Oct 06:35:20.388 Started modeling scenario.
2022-31-Oct 06:35:20.389 Validated run-time arguments.
2022-31-Oct 06:35:20.392 Connected to scenario database. Path = /Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite.
2022-31-Oct 06:35:20.399 Dropped pre-existing result tables from database.
2022-31-Oct 06:35:20.399 Verified that transmission modeling is not enabled.
2022-31-Oct 06:35:20.482 Created parameter views and indices.
2022-31-Oct 06:35:20.487 Created temporary tables.
2022-31-Oct 06:35:22.241 Executed core database queries.
2022-31-Oct 06:35:22.245 Defined dimensions.
2022-31-Oct 06:35:22.264 Defined demand variables.
2022-31-Oct 06:35:22.277 Defined storage variables.
2022-31-Oct 06:35:22.279 Defined capacity variables.
2022-31-Oct 06:35:25.271 Defined activity variables.
2022-31-Oct 06:35:25.275 Defined costing variables.
2022-31-Oct 06:35:25.276 Defined reserve margin variables.
2022-31-Oct 06:35:25.302 Defined emissi

INFEASIBLE_OR_UNBOUNDED::TerminationStatusCode = 6

In [114]:
typeof(model)

Model

In [106]:
MOI.compute_conflict!(model.moi_backend.optimizer.model)
MOI.get(model.moi_backend, Gurobi.ConstraintConflictStatus(), con1.index)



IIS runtime: 0.02 seconds (0.00 work units)


LoadError: Gurobi Error 10015: Cannot compute IIS on a feasible model